$\newcommand{\QQ}{\mathbb{Q}}\newcommand{\CC}{\mathbb{C}}\newcommand{\VV}{\mathbb{V}}\newcommand{\RR}{\mathbb{R}}$

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.13.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


## Parametric Gröbner bases in Oscar

Suppose that we have a parametric polynomial system consisting of polynomials $f_1,\ldots,f_s$ in variables $x_1,\ldots,x_n$ and parameters $u_1,\ldots,u_m$. In many applications, one would like to say something about the roots of the system for **all** (or **almost all**) values of the parameters. 

One way to do this, is to work with a *parametric Gröbner basis*. The idea is that we form the ideal $I=\langle f_1,\ldots,f_s\rangle$ in the parametric polynomial ring $\CC(u_1,\ldots,u_m)[x_1,\ldots,x_n]$, where $\CC(u_1,\ldots,u_m)$ is the field of rational functions in the parameters.

If we compute a Gröbner basis $G=\{g_1,\ldots,g_r\}$ of $I$ in this ring, then this will **specialize** for almost all parameter values $u\in\CC^m$, in the sense that $G_u:=\{g_{1,u},\ldots,g_{r,u}\}$ is a Gröbner basis for $I_u:=\langle f_{1,u},\ldots,f_{s,u}\rangle\subseteq\CC[x_1,\ldots,x_n]$, where subscript $u$ indicates that we have plugged in the specific value $u$ of the parameters.

More specifically, a *sufficient* condition for $G$ to specialize for a choice of parameters $u\in\CC^m$ is that the following three conditions are satisfied:

1. The leading coefficients of $g_1,\ldots,g_r$ don't vanish for $u$.
2. If we express each $g_i$ as a polynomial linear combination of the $f_i$'s (e.g., by computing a *transformation matrix* along with the Gröbner basis), then no denominators vanish for $u$.
3. If we express each $f_i$ as a polynomial linear combination of the $g_i$'s (e.g., using multivariate polynomial division), then no denominators vanish for $u$.


The following commands in Oscar are useful for doing this:

* To set up the field $\QQ(u_1,\ldots,u_m)$ we first form the polynomial ring $\QQ[u_1,\ldots,u_m]$ as usual, and then turn it into a field of rational functions with the command `fraction_field`.

* To compute a Gröbner basis with a transformation matrix, we use the command `groebner_basis_with_transformation_matrix`. 
<br><br> 
**Note:** It is important that you use this command to both get the Gröbner basis *and* the transformation matrix. If you use a Gröbner basis from another command (e.g., `groebner_basis`), then it might differ up to some scalar multiple, and it wouldn't be compatible with the transformation matrix obtained from `groebner_basis_with_transformation_matrix`.

* There are many ways to carry out polynomial division. One option is to use our "homemade" implementation from Week 1 (included below). 
<br><br> 
Another option is to use the built-in command `reduce_with_quotients` (but note that this is based on another division algorithm than the one we used in Week 1, and therefore sometimes gives different answers, that still satisfy all the relevant properties). 
<br><br>
A third option is to use `divrem`, but then have to specify the monomial order according to an older, outdated system, where you do it in the definition of the ring.

In [2]:
function multivariate_division(f::MPolyRingElem, F::Vector{<:MPolyRingElem}; ordering::MonomialOrdering)
    LT = (h -> leading_term(h,ordering=ordering))
    s = length(F)
    Q = [zero(parent(f)) for i=1:s]
    r = 0
    p = f
    while p != 0
        division_occured = false
        for i = 1:s
            division_occured, quotient = divides( LT(p), LT(F[i]) )
            if division_occured 
                Q[i] = Q[i] + quotient
                p = p - quotient*F[i]
                break
            end
        end
        if !(division_occured)
            r = r + LT(p)
            p = p - LT(p)
        end
    end
    return Q,r
end;

##  Exercise 20

We begin by setting up the fraction field:

In [3]:
T, (k1,k2,k3,k4,k5,k6) = polynomial_ring(QQ, ["k1","k2","k3","k4","k5","k6"])
K = fraction_field(T)

Fraction field
  of multivariate polynomial ring in 6 variables over QQ

...and then the polynomial ring:

In [4]:
R, (x5,x4,x3,x2,x1) = polynomial_ring(K, ["x5","x4","x3","x2","x1"])

(Multivariate polynomial ring in 5 variables over fraction field, AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.Frac{QQMPolyRingElem}}[x5, x4, x3, x2, x1])

We now form the polynomial ideal:

In [5]:
f1 = -k4*x2*x3 + k3*x4 + k5*x5
f2 = k1*x1*x3 - k2*x4 - k3*x4
f3 = k4*x2*x3 - k5*x5 - k6*x5

F = [f1,f2,f3]

I = ideal(F)

ideal(k5*x5 + k3*x4 - k4*x3*x2, (-k2 - k3)*x4 + k1*x3*x1, (-k5 - k6)*x5 + k4*x3*x2)

Since we want to analyze to what extent the Gröbner basis specializes in the end, we use the `groebner_basis_with_transformation_matrix` command, and save the transformation matrix $M$ for later.

In [6]:
G, M = groebner_basis_with_transformation_matrix(ideal(F), ordering=lex(R),complete_reduction=true)

(Gröbner basis with elements
1 -> (k2*k4*k6 + k3*k4*k6)*x3*x2 + (-k1*k3*k5 - k1*k3*k6)*x3*x1
2 -> (k2 + k3)*x4 - k1*x3*x1
3 -> (k2*k6 + k3*k6)*x5 - k1*k3*x3*x1
with respect to the ordering
lex([x5, x4, x3, x2, x1]), [-k2*k5-k2*k6-k3*k5-k3*k6 0 -k2-k3; -k3*k5-k3*k6 -1 -k3; -k2*k5-k3*k5 0 -k2-k3])

Let us check that the Gröbner basis works for all positive values of the parameters $k_1,\ldots,k_6$. Remember from above that we need to check three things.

**1) The leading terms:**  It is easy to see that the leading coefficients are strictly positive (and in particular nonzero) for all $k_1,\ldots,k_6>0$.

We can see this by inspection, but we can also use the `leading_coefficient` command (combined with the `map` command) as follows:

In [7]:
map(g->leading_coefficient(g,ordering=lex(R)), G)

3-element Vector{AbstractAlgebra.Generic.Frac{QQMPolyRingElem}}:
 k2*k4*k6 + k3*k4*k6
 k2 + k3
 k2*k6 + k3*k6

**2) Denominators for $g_i$'s expressed in terms of $f_i$'s:** We check the transformation matrix, and see that all of the entries have 1 as denominator, which of course is nonzero regardless of what values of the parameters $k_1,\ldots,k_6$ we plug in.

In [8]:
M

[-k2*k5 - k2*k6 - k3*k5 - k3*k6    0   -k2 - k3]
[                -k3*k5 - k3*k6   -1        -k3]
[                -k2*k5 - k3*k5    0   -k2 - k3]

**3) Denominators for $f_i$ expressed in terms of $g_i$'s:** We apply our home-made implementation of the multivariate division algorithm to each $f_i$ in the original system, and see that all denominators that appear in the quotient are nonzero.

In [9]:
# Loop through all polynomials f
for f in F
    Q,r = multivariate_division(f, elements(G), ordering=lex(R))
    display(Q)
end

3-element Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.Frac{QQMPolyRingElem}}}:
 -1//(k2*k6 + k3*k6)
 k3//(k2 + k3)
 k5//(k2*k6 + k3*k6)

3-element Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.Frac{QQMPolyRingElem}}}:
 0
 -1
 0

3-element Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.Frac{QQMPolyRingElem}}}:
 1//(k2*k6 + k3*k6)
 0
 (-k5 - k6)//(k2*k6 + k3*k6)

In [10]:
# Alternatively, use the built-in command reduce_with_quotients
for f in F
    Q,r = reduce_with_quotients(f, elements(G), ordering=lex(R))
    display(Q)
end

[-1//(k2*k6 + k3*k6)   k3//(k2 + k3)   k5//(k2*k6 + k3*k6)]

[0   -1   0]

[1//(k2*k6 + k3*k6)   0   (-k5 - k6)//(k2*k6 + k3*k6)]

### Part (a)

If we look carefully at the Gröbner basis we found above, the first polynomial describes how $x_1$ and $x_2$ are related. There is also a factor $x_3$ in both terms, but since we're only interested in solutions where $x_1,x_2,x_3,x_4,x_5>$, we can safely divide by this factor, and get a polynomial

$p=(k_2k_4k_6 + k_3k_4k_6)x_2 + (-k_1k_3k_5 - k_1k_3k_6)x_1$

that vanishes for all positive solutions. Geometrically, it means that for any positive solution $(x_1,x_2,x_3,x_4,x_5,x_6)$ of the system, the pair $(x_1,x_2)$ lies on a line in $\RR^2$ through the origin.

In [11]:
G

Gröbner basis with elements
1 -> (k2*k4*k6 + k3*k4*k6)*x3*x2 + (-k1*k3*k5 - k1*k3*k6)*x3*x1
2 -> (k2 + k3)*x4 - k1*x3*x1
3 -> (k2*k6 + k3*k6)*x5 - k1*k3*x3*x1
with respect to the ordering
lex([x5, x4, x3, x2, x1])

In [12]:
p = (1//x3)*G[1]

(k2*k4*k6 + k3*k4*k6)*x2 + (-k1*k3*k5 - k1*k3*k6)*x1

### Part (b)

Here is one possible strategy:
* Make several experiments with constant environmental factors (such as temperature), so that the unknown values of $k_1,\ldots,k_6$ stay constant, but different total concentrations. In all cases, allow the system to reach equilibrium, and then measure the concentrations $(x_1,x_2)$.
* Plot your observations of $(x_1,x_2)$ in $\mathbb{R}^2$, and check whether they seem to approximately lie on a straight line through the origin (you can use the statistical theory of linear regression or tools such as singular value decomposition to make this check more systematic).
* Repeat the analysis above at different environmental conditions, and check whether thre data still ends up lying on straight lines (but with different slopes).

<div class="alert alert-block alert-info"> <strong>Read more:</strong> The idea explored in this exercise is the foundation for a method for model selection presented in the paper <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2599844/">The Geometry of Multisite Phosphorylation</a> by Manrai and Gunawardena (2008), and further explored in <a href="https://www.pnas.org/doi/epdf/10.1073/pnas.1117073109">Parameter-free model discrimination criterion based on steady-state coplanarity</a> by Harrington, Ho, Thorne, and Stumpf (2012). </div>